In [5]:
import geopandas as gpd

# Load your Dortmund buildings file
buildings = gpd.read_parquet("D:/Work/Github_Morphotopes/data/processed_buildings/buildings_0.parquet")

# Get representative points
rep_points = buildings.representative_point()

# Turn into DataFrame for easier duplicate detection
df = gpd.GeoDataFrame(geometry=rep_points)
df["x"] = df.geometry.x
df["y"] = df.geometry.y

# Find duplicates
dupes = df[df.duplicated(subset=["x", "y"], keep=False)]

# Show the duplicates and their indexes (which match building IDs)
print(dupes)
print("Number of duplicate coordinates:", dupes.shape[0] // 2)  # divide by 2 for pairs

                             geometry              x             y
16059  POINT (386796.604 5705705.876)  386796.604448  5.705706e+06
21728  POINT (398200.173 5708923.093)  398200.173185  5.708923e+06
37837  POINT (398200.173 5708923.093)  398200.173185  5.708923e+06
48632  POINT (386796.604 5705705.876)  386796.604448  5.705706e+06
Number of duplicate coordinates: 2


In [6]:
rp = buildings.representative_point()
buildings["x"] = rp.x
buildings["y"] = rp.y

buildings = buildings.reset_index().rename(columns={"index": "bldg_id"})

buildings.to_file("D:/Work/Github_Morphotopes/data/processed_buildings/buildings_0.gpkg", layer="buildings_0", driver="GPKG")

In [7]:
buildings.head

<bound method NDFrame.head of         bldg_id                  OBJ_NAME OBJ_ID  OBJ_FOLIE  OBJ_OSKA  \
0             0  05913000727760010    001    ---         11         0   
1             1  05913000725420019   c001    ---         11         0   
2             2  05913000718320104    003    ---         11         0   
3             3  05913000734080011    001    ---         11         0   
4             4  05913000702410003    004    ---         11         0   
...         ...                       ...    ...        ...       ...   
203719   203719  05913000721760364   d001    ---         11         0   
203720   203720  05913000712410124    008    ---         11         0   
203721   203721  05913000707110003    005    ---         11         0   
203722   203722  05913000718880159    004    ---         11         0   
203723   203723  05913000718290038    001    ---         11         0   

       ELGEO_ID  SCHLUESSEL                   NAME              UUID  \
0           ---      

In [11]:
import geopandas as gpd
import pandas as pd
region_mapping = pd.read_parquet("D:/Work/Github_Morphotopes/data/cadastre_id_to_region.parquet")
b = gpd.read_parquet("D:/Work/Github_Morphotopes/data/processed_buildings/buildings_0.parquet")
rp = b.representative_point()
rp_xy = pd.DataFrame({"x": rp.x, "y": rp.y}, index=b.index)

# 1) Which buildings share the same representative point?
dupe_rp = rp_xy[rp_xy.duplicated(["x","y"], keep=False)].sort_values(["x","y"])
print(dupe_rp.head())

# 2) Compare with region_mapping coords for those same IDs
ids = dupe_rp.index.unique()
print(region_mapping.loc[ids, ["x","y"]].head())

                   x             y
16059  386796.604448  5.705706e+06
48632  386796.604448  5.705706e+06
21728  398200.173185  5.708923e+06
37837  398200.173185  5.708923e+06
                   x             y
16059  386803.609337  5.705701e+06
48632  386803.824746  5.705702e+06
21728  398200.221097  5.708923e+06
37837  398199.546949  5.708922e+06
